In [1]:
import os

In [2]:
%pwd

'e:\\OneDrive\\Documents\\End-to-End-MLP-with-MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\OneDrive\\Documents\\End-to-End-MLP-with-MLFlow'

**Actualizo los data paths nuevamente:**

* los de mi config.yaml para Data Transformacion

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

* importo las dos mismas librerias igual que en validation.


In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

* Cargo la lectura de los .yaml y el configuracionManager (esa estructura es comun en casi todos lso pasos como ven) 

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig: #<--- meciono el get data transformation config 
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

* **Como  la data ya estará limpia no nos enfocaremos en procesos de limpieza o mejora tipo encoding en este punto si no que en como podemos hacer la pipeline integration a MLflow, asi que avanzaremos directo a entreno y testeo.**

In [9]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    ## Note: Aquí podemos agregar diferentes tecnicas de tranformacion de data  como Scaler, PCA y todo eso.
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up


    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False) #<--- mandara a la ruta para almacenar el train.csv en el data transformation folder
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False) #<--- mandara a la ruta para almacenar el test.csv en el data transformation folder

        logger.info("Splited data into training and test sets") #<--- logeamos el proceso en terminal y en el log
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)
        

* **corremos el train,test split**

In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2023-09-28 22:03:02,374: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-28 22:03:02,376: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-28 22:03:02,380: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-28 22:03:02,381: INFO: common: created directory at: artifacts]
[2023-09-28 22:03:02,383: INFO: common: created directory at: artifacts/data_transformation]
[2023-09-28 22:03:02,507: INFO: 279802063: Splited data into training and test sets]
[2023-09-28 22:03:02,508: INFO: 279802063: (1199, 12)]
[2023-09-28 22:03:02,509: INFO: 279802063: (400, 12)]
(1199, 12)
(400, 12)
